In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import dgl
import dgl.data
import dgl.nn as gnn

from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

Задача: предсказать какую-нибудь характеристику на уровне целого графа

Используем синтетический датасет GIN. 
Он содержит некоторое кол-во графов, у каждого графа есть 
а) фичи на узлах и 
б) метка на весь граф 

In [6]:
dataset = dgl.data.GINDataset('PROTEINS', self_loop=True)
print('Node feature dimensionality:', dataset.dim_nfeats)
print('Number of graph categories:', dataset.gclasses)


Node feature dimensionality: 3
Number of graph categories: 2


Проблема: нужно уметь разбивать датасет на мини-батчи. Решение: `GraphDataLoader`, который работает со стандартными сэмплерами из `torch`.

Каждый элемент в датасете представляет собой пару (граф, метка). `GraphDataLoader` при итерации по нему возвращает два объекта: объедененный граф для батча и вектор с метками для каждого графа из минибатча. 

Объединенный граф состоит из нескольких графов, объединенных в один в виде несвязных компонент. Фичи узлов и ребер сохраняются. Это такой же граф, как и обычный `DGLGraph`, но содержит доп. информацию для восстановления исходных графов. Развернуть графы назад можно с помощью метода `unbatch`

In [11]:
num_examples = len(dataset)
num_train = int(num_examples * .8)

train_sampler = SubsetRandomSampler(torch.arange(num_train))
test_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))

train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=5, drop_last=False)
test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=5, drop_last=False)


In [14]:
it = iter(train_dataloader)
batched_graph, labels = next(it)
print('Number of nodes for each graph element in the batch:',
      batched_graph.batch_num_nodes())
print('Number of edges for each graph element in the batch:',
      batched_graph.batch_num_edges())



Number of nodes for each graph element in the batch: tensor([ 16,  14,  27,  31, 125])
Number of edges for each graph element in the batch: tensor([ 80,  64, 129, 145, 553])


In [15]:
# Recover the original graph elements from the minibatch
graphs = dgl.unbatch(batched_graph)
print('The original graphs in the minibatch:')
print(graphs)


The original graphs in the minibatch:
[Graph(num_nodes=16, num_edges=80,
      ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=14, num_edges=64,
      ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=27, num_edges=129,
      ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=31, num_edges=145,
      ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=125, num_edges=553,
      ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={})]


Workflow:
1. Создаем как обычно двухслойную сеть. На вход придет батч-граф. 
2. Дальше нужно сагрегировать представления узлов (и возможно ребер) чтобы получить представление графа в целом (самый простой вариант - усреднить с помощью `dgl.mean_nodes()`). Этот процесс называют `readout`. `DGL` предоставляет набор функций, которые могут работать с батч-графами и получать для них представление. 

In [21]:
class GCN(nn.Module):

    def __init__(self, n_inputs, n_hidden, num_classes):
        super().__init__()
        self.conv1 = gnn.GraphConv(n_inputs, n_hidden)
        self.conv2 = gnn.GraphConv(n_hidden, num_classes)

    def forward(self, G, features):
        out = F.relu(self.conv1(G, features))
        out = self.conv2(G, out)
        G.ndata['h'] = out
        return dgl.mean_nodes(G, 'h')


In [25]:
n_inputs, n_hidden, n_out = dataset.dim_nfeats, 16, dataset.gclasses
model = GCN(n_inputs, n_hidden, n_out)
optimizer = torch.optim.Adam(model.parameters(), lr=.01)

n_epochs = 20
for epoch in range(n_epochs):
    for batched_graph, labels in train_dataloader:
        pred = model(batched_graph, batched_graph.ndata['attr'].float())
        loss = F.cross_entropy(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

num_correct = 0
num_tests = 0
for batched_graph, labels in test_dataloader:
    pred = model(batched_graph, batched_graph.ndata['attr'].float())
    num_correct += (pred.argmax(1) == labels).sum().item()
    num_tests += len(labels)

print('Test accuracy:', num_correct / num_tests)


Test accuracy: 0.24663677130044842
